# Random Patch Sampler (3D)

In [1]:
# ms-python.python added
import os

In [18]:
import glob, shutil
import cv2
import json
import numpy as np
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch, random_crop_batch_3D, patch_sampler_3D, volume_loader, sampler_3D
from datetime import datetime
from pprint import pprint
from tqdm.notebook import trange

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# set parameters
# crop size
zdim = 20
ydim = 256
xdim = 256
crop_per_image = 200
seed = 100
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
output_folder = timestamp + '_' + str(crop_per_image) + 'x'
print("Crop Size z: {}".format(zdim))
print("Crop Size y: {}".format(ydim))
print("Crop Size x: {}".format(xdim))
print('Crop per Image: {}'.format(crop_per_image))
print('Seed: {}'.format(seed))
print('timestamp: {}'.format(timestamp))

Crop Size z: 20
Crop Size y: 256
Crop Size x: 256
Crop per Image: 200
Seed: 100
timestamp: 2020_01_10_17_23


In [4]:
# Prepare the training dataset
# Specify the input folder
# mainpath = '/Volumes/LaCie_DataStorage/PerlmutterData/dl_seg_project_raw'

def listdir_nohidden(dir):
    return [file for file in os.listdir(dir) if not file.startswith('.')]

working_folder = 'dl_seg_project_raw_3D_original'

mainpath = os.path.join('D:', 'PerlmutterData', working_folder)
img_seg_dir = 'data_Amira'
labeltypes = listdir_nohidden(os.path.join(mainpath, img_seg_dir))
ipdir = os.path.join(mainpath, img_seg_dir)
print(ipdir)
print(labeltypes)

D:PerlmutterData\dl_seg_project_raw_3D_original\data_Amira
['autophagosome', 'cell_membrane', 'mitochondria', 'nucleus']


In [5]:
# Create output folder
print("Check the output folder:")
dir_checker('data_crop_3D', mainpath)
dir_checker(output_folder, os.path.join(mainpath, 'data_crop_3D'))
opdir = os.path.join(mainpath, 'data_crop_3D', output_folder)
dir_checker('pars', opdir)
dir_checker('images', opdir)
dir_checker('labels', opdir)
print(opdir)

Check the output folder:
data_crop_3D exists in D:PerlmutterData\dl_seg_project_raw_3D_original
2020_01_10_15_41_20x does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D
pars does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x
images does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x
labels does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x
D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x


In [6]:
# create parameter
pars = dict(
            mainpath = mainpath,
            input_path = ipdir,
            output_path = opdir, 
            zdim = zdim,
            ydim = ydim,
            xdim = xdim, 
            crop_per_image = crop_per_image,
            )

with open(os.path.join(opdir, 'pars', 'pars_' + timestamp + '.json'), 'w') as outfile:
    json.dump(pars, outfile, indent=4)

In [7]:
data_path = [os.path.join(ipdir, labeltype) for labeltype in labeltypes ]
print("Path: {}".format(data_path))

Path: ['D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\autophagosome', 'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane', 'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\mitochondria', 'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\nucleus']


In [15]:
# create list for filenames
ipimglist = {}
iplabellist = {}

for labeltype in labeltypes:
    imglist_tmp = glob.glob(os.path.join(ipdir, labeltype, 'images', '*'), recursive=True)
    labellist_tmp = glob.glob(os.path.join(ipdir, labeltype, 'labels', '*'), recursive=True)
    ipimglist[labeltype] = imglist_tmp
    iplabellist[labeltype] = labellist_tmp

In [16]:
pprint(ipimglist)

{'autophagosome': ['D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\autophagosome\\images\\data_d00_batch01_loc01_label03_stack01',
                   'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\autophagosome\\images\\data_d07_batch01_loc01_label03_stack01',
                   'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\autophagosome\\images\\data_d14_batch01_loc01_label03_stack01',
                   'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\autophagosome\\images\\data_d17_batch01_loc01_label03_stack01',
                   'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\autophagosome\\images\\data_d21_batch01_loc01_label03_stack01'],
 'cell_membrane': ['D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d00_batch01_loc01_label01_stack01',
                   'D:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d07_batch01_l

In [11]:
for labeltype in labeltypes:
    tmp_imgcount_3D_list = listdir_nohidden(os.path.join(ipdir, labeltype, 'images'))
    tmp_labelcount_3D_list = listdir_nohidden(os.path.join(ipdir, labeltype, 'labels'))
    print(labeltype)
    print("Input Image Counts(3D): {}".format(len(tmp_imgcount_3D_list)))
    print("Input Label Counts(3D): {}".format(len(tmp_labelcount_3D_list)))

autophagosome
Input Image Counts(3D): 5
Input Label Counts(3D): 5
cell_membrane
Input Image Counts(3D): 6
Input Label Counts(3D): 6
mitochondria
Input Image Counts(3D): 5
Input Label Counts(3D): 5
nucleus
Input Image Counts(3D): 6
Input Label Counts(3D): 6


In [14]:
imgtypes = ['images', 'labels']
for imgtype in imgtypes:
    dir_checker(imgtype, opdir)
    for labeltype in labeltypes:
        dir_checker(labeltype, os.path.join(opdir, imgtype))

images exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x
autophagosome does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images
cell_membrane does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images
mitochondria does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images
nucleus does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images
labels exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x
autophagosome does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels
cell_membrane does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels
mitochondria does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data

In [40]:
# Batch Random Crop
for labeltype in labeltypes:     
    print("Croping the images from group {}...".format(labeltype))
    random_crop_batch_3D(ipimglist = ipimglist, 
                      iplabellist = iplabellist, 
                      opfolder = opdir, 
                      label = labeltype, 
                      crop_size = [zdim, ydim, xdim], 
                      crop_per_image = crop_per_image, 
                      seed = seed)
    print("Finish")

Croping the images from group autophagosome...



(20, 883, 2297)
0001 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0001 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0002 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0002 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0003 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0003 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0004 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0004 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0005 exists in D:PerlmutterData\dl_seg_

0035 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0035 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0036 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0036 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0037 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0037 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0038 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0038 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autoph

0069 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0070 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0070 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0071 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0071 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0072 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0072 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0073 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autoph

0104 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0104 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0105 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0105 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0106 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0106 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autophagosome
0107 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\autophagosome
0107 does not exist in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\autoph

...

0039 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0040 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0040 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0041 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0041 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0042 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0042 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0043 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0043 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\

0078 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0078 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0079 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0079 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0080 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0080 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0081 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0081 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0082 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\

0117 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0117 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0118 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0118 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0119 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0119 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0120 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0120 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0121 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\

0154 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0154 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0155 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0155 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0156 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0156 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0157 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0157 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0158 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\

0193 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0193 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0194 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0194 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0195 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0195 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0196 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\nucleus
0196 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\labels\nucleus
0197 exists in D:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_10_15_41_20x\images\